In [1]:
# Import dependencies
import pandas as pd
import re
import numpy as np

In [2]:
# CSV translated from .xlsx (Excel workbook) file downloaded from:
# https://www.wake.gov/departments-government/tax-administration/data-files-statistics-and-reports/real-estate-property-data-files
housing_df = pd.read_csv("resources/original/wake_county_residential_data_original.csv", 
                         usecols=["Street_Number", "Street_Prefix", "Street_Name", "Street_Type", "Street_Suffix", 
                                  "Planning_Jurisdiction", "Zoning", "Deeded_Acreage", "Assessed_Building_Value", 
                                  "Assessed_Land_Value", "BILLING_CLASS", "Year_Built", "UNITS", "HEATED_AREA", 
                                  "Year_of_Addition", "Remodeled_Year", "DESIGN_STYLE", "BATH", "PHYSICAL_CITY", 
                                  "PHYSICAL_ZIP_CODE"],
                         dtype={"Street_Suffix": str}) # Street_Suffix has conflicting types; set to string to avoid error
housing_df = housing_df.fillna("OTHER")

housing_df.head()

,Street_Number,Street_Prefix,Street_Name,Street_Type,Street_Suffix,Planning_Jurisdiction,Zoning,Deeded_Acreage,Assessed_Building_Value,Assessed_Land_Value,BILLING_CLASS,Year_Built,UNITS,HEATED_AREA,Year_of_Addition,Remodeled_Year,DESIGN_STYLE,BATH,PHYSICAL_CITY,PHYSICAL_ZIP_CODE
0,1506,OTHER,WAKE FOREST,RD,OTHER,RA,NX-3,0.32,0,"112,906",1,0,0,0,0,0,OTHER,OTHER,RALEIGH,27604.0
1,6012,OTHER,TRIANGLE,DR,OTHER,RA,IX-3,2.00,"524,503","392,040",1,1979,0,4500,0,0,A,I,RALEIGH,27617.0
2,6012,OTHER,TRIANGLE,DR,OTHER,RA,IX-3,2.00,"524,503","392,040",1,1989,0,4008,0,0,A,I,RALEIGH,27617.0
3,1601,OTHER,WAKE FOREST,RD,OTHER,RA,IX-3,1.38,"374,128","245,076",1,1993,0,8500,0,0,A,I,RALEIGH,27604.0
4,1831,OTHER,CAPITAL,BLVD,OTHER,RA,IX-3,1.11,"238,511","578,774",1,1968,0,3088,0,0,A,I,RALEIGH,27604.0


In [3]:
# Keep only residential zones (begin with "R-" or "R" and a number)
housing_df = housing_df[housing_df["Zoning"].str.contains('^R-|^R\d+')]

# Keep only the number of each zone
housing_df["Zoning"] = housing_df["Zoning"].str.extract(r'(\d+)\Z', expand=True)

housing_df.head()

,Street_Number,Street_Prefix,Street_Name,Street_Type,Street_Suffix,Planning_Jurisdiction,Zoning,Deeded_Acreage,Assessed_Building_Value,Assessed_Land_Value,BILLING_CLASS,Year_Built,UNITS,HEATED_AREA,Year_of_Addition,Remodeled_Year,DESIGN_STYLE,BATH,PHYSICAL_CITY,PHYSICAL_ZIP_CODE
5,609,OTHER,BAILEY,DR,OTHER,RA,10,13.83,0,"394,155",3,0,0,0,0,0,OTHER,OTHER,RALEIGH,27610.0
8,2457,OTHER,BERTIE,DR,OTHER,RA,4,0.21,"134,321","115,000",2,1964,1,1828,0,0,I,C,RALEIGH,27610.0
9,2848,OTHER,PROVIDENCE,RD,OTHER,RA,4,0.46,"114,933","45,000",2,1970,1,1240,0,0,A,A,RALEIGH,27610.0
10,409,S,LAKESIDE,DR,OTHER,RA,4,0.43,"132,624","90,000",2,1999,1,1037,0,0,A,C,RALEIGH,27606.0
11,540,OTHER,MARSHBURN,RD,OTHER,WE,3,0.46,"118,723","32,000",2,1900,1,2261,0,0,A,C,WENDELL,27591.0


In [4]:
# Keep only houses with an individual's billing class (as opposed to corporations, HOA, etc.)
housing_df = housing_df.loc[housing_df["BILLING_CLASS"] == 2]

# Drop the column
housing_df = housing_df.drop(columns=["BILLING_CLASS"])

housing_df.head()

,Street_Number,Street_Prefix,Street_Name,Street_Type,Street_Suffix,Planning_Jurisdiction,Zoning,Deeded_Acreage,Assessed_Building_Value,Assessed_Land_Value,Year_Built,UNITS,HEATED_AREA,Year_of_Addition,Remodeled_Year,DESIGN_STYLE,BATH,PHYSICAL_CITY,PHYSICAL_ZIP_CODE
8,2457,OTHER,BERTIE,DR,OTHER,RA,4,0.21,"134,321","115,000",1964,1,1828,0,0,I,C,RALEIGH,27610.0
9,2848,OTHER,PROVIDENCE,RD,OTHER,RA,4,0.46,"114,933","45,000",1970,1,1240,0,0,A,A,RALEIGH,27610.0
10,409,S,LAKESIDE,DR,OTHER,RA,4,0.43,"132,624","90,000",1999,1,1037,0,0,A,C,RALEIGH,27606.0
11,540,OTHER,MARSHBURN,RD,OTHER,WE,3,0.46,"118,723","32,000",1900,1,2261,0,0,A,C,WENDELL,27591.0
13,1612,OTHER,BENNETT,ST,OTHER,RA,4,0.76,0,"202,500",0,0,0,0,0,OTHER,OTHER,RALEIGH,27604.0


In [5]:
# Keep only lots with houses (not empty lots)
housing_df = housing_df.loc[housing_df["Year_Built"] != 0]

# Create column of house age
housing_df["AGE"] = 2023 - housing_df["Year_Built"]

# Drop year built column
housing_df = housing_df.drop(columns=["Year_Built"])

housing_df.head()

,Street_Number,Street_Prefix,Street_Name,Street_Type,Street_Suffix,Planning_Jurisdiction,Zoning,Deeded_Acreage,Assessed_Building_Value,Assessed_Land_Value,UNITS,HEATED_AREA,Year_of_Addition,Remodeled_Year,DESIGN_STYLE,BATH,PHYSICAL_CITY,PHYSICAL_ZIP_CODE,AGE
8,2457,OTHER,BERTIE,DR,OTHER,RA,4,0.21,"134,321","115,000",1,1828,0,0,I,C,RALEIGH,27610.0,59
9,2848,OTHER,PROVIDENCE,RD,OTHER,RA,4,0.46,"114,933","45,000",1,1240,0,0,A,A,RALEIGH,27610.0,53
10,409,S,LAKESIDE,DR,OTHER,RA,4,0.43,"132,624","90,000",1,1037,0,0,A,C,RALEIGH,27606.0,24
11,540,OTHER,MARSHBURN,RD,OTHER,WE,3,0.46,"118,723","32,000",1,2261,0,0,A,C,WENDELL,27591.0,123
14,8712,W,LAKE,CT,OTHER,RA,2,0.96,"358,290","140,000",1,3770,0,0,A,J,RALEIGH,27613.0,52


In [6]:
# Keep only lots with 1 unit (i.e. not apartments), and with heated area (livable square footage)
housing_df = housing_df.loc[housing_df["UNITS"] == 1]
housing_df = housing_df.loc[housing_df["HEATED_AREA"] != 0]

# Drop units column
housing_df = housing_df.drop(columns="UNITS")

housing_df.head()

,Street_Number,Street_Prefix,Street_Name,Street_Type,Street_Suffix,Planning_Jurisdiction,Zoning,Deeded_Acreage,Assessed_Building_Value,Assessed_Land_Value,HEATED_AREA,Year_of_Addition,Remodeled_Year,DESIGN_STYLE,BATH,PHYSICAL_CITY,PHYSICAL_ZIP_CODE,AGE
8,2457,OTHER,BERTIE,DR,OTHER,RA,4,0.21,"134,321","115,000",1828,0,0,I,C,RALEIGH,27610.0,59
9,2848,OTHER,PROVIDENCE,RD,OTHER,RA,4,0.46,"114,933","45,000",1240,0,0,A,A,RALEIGH,27610.0,53
10,409,S,LAKESIDE,DR,OTHER,RA,4,0.43,"132,624","90,000",1037,0,0,A,C,RALEIGH,27606.0,24
11,540,OTHER,MARSHBURN,RD,OTHER,WE,3,0.46,"118,723","32,000",2261,0,0,A,C,WENDELL,27591.0,123
14,8712,W,LAKE,CT,OTHER,RA,2,0.96,"358,290","140,000",3770,0,0,A,J,RALEIGH,27613.0,52


In [7]:
# Create site address column from street number, prefix, name, type, and suffix
housing_df["SITE_ADDRESS"] = housing_df["Street_Number"].astype(str) + " " + housing_df["Street_Prefix"] + " " + housing_df["Street_Name"] + " " + housing_df["Street_Type"] + " " + housing_df["Street_Suffix"]

# If value is "OTHER" (was NaN), replace with nothing
housing_df["SITE_ADDRESS"] = housing_df["SITE_ADDRESS"].str.replace(r'\s?OTHER', '', regex=True)

# Remove extra spaces
housing_df["SITE_ADDRESS"] = housing_df["SITE_ADDRESS"].str.replace(r'\s+', ' ', regex=True)

# Drop street number, prefix, name, type, and suffix columns
housing_df = housing_df.drop(columns=["Street_Number", "Street_Prefix", "Street_Name", "Street_Type", "Street_Suffix"])

housing_df.head()

,Planning_Jurisdiction,Zoning,Deeded_Acreage,Assessed_Building_Value,Assessed_Land_Value,HEATED_AREA,Year_of_Addition,Remodeled_Year,DESIGN_STYLE,BATH,PHYSICAL_CITY,PHYSICAL_ZIP_CODE,AGE,SITE_ADDRESS
8,RA,4,0.21,"134,321","115,000",1828,0,0,I,C,RALEIGH,27610.0,59,2457 BERTIE DR
9,RA,4,0.46,"114,933","45,000",1240,0,0,A,A,RALEIGH,27610.0,53,2848 PROVIDENCE RD
10,RA,4,0.43,"132,624","90,000",1037,0,0,A,C,RALEIGH,27606.0,24,409 S LAKESIDE DR
11,WE,3,0.46,"118,723","32,000",2261,0,0,A,C,WENDELL,27591.0,123,540 MARSHBURN RD
14,RA,2,0.96,"358,290","140,000",3770,0,0,A,J,RALEIGH,27613.0,52,8712 W LAKE CT


In [8]:
# Change assessed building and land values to integers
housing_df = housing_df.replace(",", "", regex=True)
housing_df = housing_df.astype({"Assessed_Building_Value": "int", "Assessed_Land_Value": "int"})

# Create total value column to combine house and land values
housing_df["TOTAL_VALUE"] = housing_df["Assessed_Building_Value"] + housing_df["Assessed_Land_Value"]

# Drop assessed value columns
housing_df = housing_df.drop(columns=["Assessed_Building_Value", "Assessed_Land_Value"])

housing_df.head()

,Planning_Jurisdiction,Zoning,Deeded_Acreage,HEATED_AREA,Year_of_Addition,Remodeled_Year,DESIGN_STYLE,BATH,PHYSICAL_CITY,PHYSICAL_ZIP_CODE,AGE,SITE_ADDRESS,TOTAL_VALUE
8,RA,4,0.21,1828,0,0,I,C,RALEIGH,27610.0,59,2457 BERTIE DR,249321
9,RA,4,0.46,1240,0,0,A,A,RALEIGH,27610.0,53,2848 PROVIDENCE RD,159933
10,RA,4,0.43,1037,0,0,A,C,RALEIGH,27606.0,24,409 S LAKESIDE DR,222624
11,WE,3,0.46,2261,0,0,A,C,WENDELL,27591.0,123,540 MARSHBURN RD,150723
14,RA,2,0.96,3770,0,0,A,J,RALEIGH,27613.0,52,8712 W LAKE CT,498290


In [9]:
# Create remodel/addition column for houses that have remodelled or added an addition (true/false) in (1/0) form
housing_df["REMODEL_ADDITION"] = np.where((housing_df["Year_of_Addition"] != 0) | (housing_df["Remodeled_Year"] != 0), 1, 0)

# Drop year of addition and remodeled year columns
housing_df = housing_df.drop(columns=["Year_of_Addition", "Remodeled_Year"])

housing_df.head()

,Planning_Jurisdiction,Zoning,Deeded_Acreage,HEATED_AREA,DESIGN_STYLE,BATH,PHYSICAL_CITY,PHYSICAL_ZIP_CODE,AGE,SITE_ADDRESS,TOTAL_VALUE,REMODEL_ADDITION
8,RA,4,0.21,1828,I,C,RALEIGH,27610.0,59,2457 BERTIE DR,249321,0
9,RA,4,0.46,1240,A,A,RALEIGH,27610.0,53,2848 PROVIDENCE RD,159933,0
10,RA,4,0.43,1037,A,C,RALEIGH,27606.0,24,409 S LAKESIDE DR,222624,0
11,WE,3,0.46,2261,A,C,WENDELL,27591.0,123,540 MARSHBURN RD,150723,0
14,RA,2,0.96,3770,A,J,RALEIGH,27613.0,52,8712 W LAKE CT,498290,0


In [10]:
# Change planning jurisdiction to numbers for easier analysis and machine learning
housing_df["Planning_Jurisdiction"] = np.where(housing_df["Planning_Jurisdiction"] == "AN", "0", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "AP", "1", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "CA", "2", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "CL", "3", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "DU", "4", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "FV", "5", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "GA", "6", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "HS", "7", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "KN", "8", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "MO", "9", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "RA", "10", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "RD", "11", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "RO", "12", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "WC", "13", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "WE", "14", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "WF", "15", 
                                               np.where(housing_df["Planning_Jurisdiction"] == "ZB", "16", "17")))))))))))))))))
housing_df["Planning_Jurisdiction"] = housing_df["Planning_Jurisdiction"].astype("int")

# Drop houses without a planning jurisdiction (set to 17)
housing_df = housing_df.loc[housing_df["Planning_Jurisdiction"] != 17]

housing_df.head()

,Planning_Jurisdiction,Zoning,Deeded_Acreage,HEATED_AREA,DESIGN_STYLE,BATH,PHYSICAL_CITY,PHYSICAL_ZIP_CODE,AGE,SITE_ADDRESS,TOTAL_VALUE,REMODEL_ADDITION
8,10,4,0.21,1828,I,C,RALEIGH,27610.0,59,2457 BERTIE DR,249321,0
9,10,4,0.46,1240,A,A,RALEIGH,27610.0,53,2848 PROVIDENCE RD,159933,0
10,10,4,0.43,1037,A,C,RALEIGH,27606.0,24,409 S LAKESIDE DR,222624,0
11,14,3,0.46,2261,A,C,WENDELL,27591.0,123,540 MARSHBURN RD,150723,0
14,10,2,0.96,3770,A,J,RALEIGH,27613.0,52,8712 W LAKE CT,498290,0


In [11]:
# Change design style to numbers for easier analysis and machine learning
housing_df["DESIGN_STYLE"] = np.where(housing_df["DESIGN_STYLE"] == "A", "0", np.where(housing_df["DESIGN_STYLE"] == "B", "1",
                                      np.where(housing_df["DESIGN_STYLE"] == "C", "2", np.where(housing_df["DESIGN_STYLE"] == "D", "3",
                                      np.where(housing_df["DESIGN_STYLE"] == "E", "4", np.where(housing_df["DESIGN_STYLE"] == "F", "5",
                                      np.where(housing_df["DESIGN_STYLE"] == "G", "6", np.where(housing_df["DESIGN_STYLE"] == "H", "7",
                                      np.where(housing_df["DESIGN_STYLE"] == "I", "8", np.where(housing_df["DESIGN_STYLE"] == "J", "9",
                                      np.where(housing_df["DESIGN_STYLE"] == "K", "10", np.where(housing_df["DESIGN_STYLE"] == "L", "11",
                                      np.where(housing_df["DESIGN_STYLE"] == "M", "12", np.where(housing_df["DESIGN_STYLE"] == "N", "13",
                                      np.where(housing_df["DESIGN_STYLE"] == "O", "14", "15")))))))))))))))
housing_df["DESIGN_STYLE"] = housing_df["DESIGN_STYLE"].astype("int")

# Drop houses without a design style (set to 15)
housing_df = housing_df.loc[housing_df["DESIGN_STYLE"] != 15]

housing_df.head()

,Planning_Jurisdiction,Zoning,Deeded_Acreage,HEATED_AREA,DESIGN_STYLE,BATH,PHYSICAL_CITY,PHYSICAL_ZIP_CODE,AGE,SITE_ADDRESS,TOTAL_VALUE,REMODEL_ADDITION
8,10,4,0.21,1828,8,C,RALEIGH,27610.0,59,2457 BERTIE DR,249321,0
9,10,4,0.46,1240,0,A,RALEIGH,27610.0,53,2848 PROVIDENCE RD,159933,0
10,10,4,0.43,1037,0,C,RALEIGH,27606.0,24,409 S LAKESIDE DR,222624,0
11,14,3,0.46,2261,0,C,WENDELL,27591.0,123,540 MARSHBURN RD,150723,0
14,10,2,0.96,3770,0,J,RALEIGH,27613.0,52,8712 W LAKE CT,498290,0


In [12]:
# Change bath column to number count (each letter represents a number of baths) and change to float data type
housing_df["BATH"] = np.where(housing_df["BATH"] == "A", "1", np.where(housing_df["BATH"] == "B", "1.5", 
                              np.where(housing_df["BATH"] == "C", "2", np.where(housing_df["BATH"] == "D", "2.5",
                              np.where(housing_df["BATH"] == "E", "3", np.where(housing_df["BATH"] == "F", "3.5", 0))))))
housing_df["BATH"] = housing_df["BATH"].astype("float")

# Remove houses without any bathrooms (set to 0)
housing_df = housing_df.loc[housing_df["BATH"] != 0]

housing_df.head()

,Planning_Jurisdiction,Zoning,Deeded_Acreage,HEATED_AREA,DESIGN_STYLE,BATH,PHYSICAL_CITY,PHYSICAL_ZIP_CODE,AGE,SITE_ADDRESS,TOTAL_VALUE,REMODEL_ADDITION
8,10,4,0.21,1828,8,2.0,RALEIGH,27610.0,59,2457 BERTIE DR,249321,0
9,10,4,0.46,1240,0,1.0,RALEIGH,27610.0,53,2848 PROVIDENCE RD,159933,0
10,10,4,0.43,1037,0,2.0,RALEIGH,27606.0,24,409 S LAKESIDE DR,222624,0
11,14,3,0.46,2261,0,2.0,WENDELL,27591.0,123,540 MARSHBURN RD,150723,0
15,6,2,0.51,996,0,1.0,RALEIGH,27603.0,67,605 WOODLAND RD,140801,0


In [13]:
# Remove houses without a zip code
housing_df = housing_df.loc[housing_df["PHYSICAL_ZIP_CODE"] != "OTHER"]

# Change zip code to int
housing_df["PHYSICAL_ZIP_CODE"] = housing_df["PHYSICAL_ZIP_CODE"].astype("int")

housing_df.head()

,Planning_Jurisdiction,Zoning,Deeded_Acreage,HEATED_AREA,DESIGN_STYLE,BATH,PHYSICAL_CITY,PHYSICAL_ZIP_CODE,AGE,SITE_ADDRESS,TOTAL_VALUE,REMODEL_ADDITION
8,10,4,0.21,1828,8,2.0,RALEIGH,27610,59,2457 BERTIE DR,249321,0
9,10,4,0.46,1240,0,1.0,RALEIGH,27610,53,2848 PROVIDENCE RD,159933,0
10,10,4,0.43,1037,0,2.0,RALEIGH,27606,24,409 S LAKESIDE DR,222624,0
11,14,3,0.46,2261,0,2.0,WENDELL,27591,123,540 MARSHBURN RD,150723,0
15,6,2,0.51,996,0,1.0,RALEIGH,27603,67,605 WOODLAND RD,140801,0


In [14]:
# Reorder and rename columns
housing_df = housing_df[["TOTAL_VALUE", "SITE_ADDRESS", "PHYSICAL_CITY", "PHYSICAL_ZIP_CODE", "Planning_Jurisdiction", 
                         "Zoning", "Deeded_Acreage", "HEATED_AREA", "AGE", "BATH", "REMODEL_ADDITION", "DESIGN_STYLE"]]
housing_df.rename(columns={"PHYSICAL_CITY": "CITY", "PHYSICAL_ZIP_CODE": "ZIP_CODE", 
                           "Planning_Jurisdiction": "PLANNING_JURISDICTION", "Zoning": "ZONING", 
                           "Deeded_Acreage": "ACREAGE", "DESIGN_STYLE": "STYLE"
                           }, inplace=True)

housing_df.head()

,TOTAL_VALUE,SITE_ADDRESS,CITY,ZIP_CODE,PLANNING_JURISDICTION,ZONING,ACREAGE,HEATED_AREA,AGE,BATH,REMODEL_ADDITION,STYLE
8,249321,2457 BERTIE DR,RALEIGH,27610,10,4,0.21,1828,59,2.0,0,8
9,159933,2848 PROVIDENCE RD,RALEIGH,27610,10,4,0.46,1240,53,1.0,0,0
10,222624,409 S LAKESIDE DR,RALEIGH,27606,10,4,0.43,1037,24,2.0,0,0
11,150723,540 MARSHBURN RD,WENDELL,27591,14,3,0.46,2261,123,2.0,0,0
15,140801,605 WOODLAND RD,RALEIGH,27603,6,2,0.51,996,67,1.0,0,0


In [15]:
# Check for duplicates
print(housing_df.shape)
print(housing_df["SITE_ADDRESS"].nunique())

(188421, 12)
185931


In [16]:
# Drop duplicates
housing_df.drop_duplicates(subset=["SITE_ADDRESS"], keep='first', inplace=True)

In [17]:
# Check if dropping duplicates worked
print(housing_df.shape)
print(housing_df["SITE_ADDRESS"].nunique())

(185931, 12)
185931


In [18]:
# Export to csv file
housing_df.to_csv('resources/clean/housing_data_clean.csv', index=False)